# Build and Train SiameseNet with Triplet Loss

Now when we have the encodings of all faces, we can train small siamese model (even on our laptop) to distinguish whether 2 images show the same person.

In [1]:
import pandas as pd
import numpy as np

celeb_data = pd.read_csv('identity_CelebA.txt', sep=" ", header=None)
celeb_data.columns = ["image", "label"]
partition = pd.read_csv('list_eval_partition.csv')
df_train = celeb_data[partition.partition==0]
df_valid = celeb_data[partition.partition==1]
df_test = celeb_data[partition.partition==2]

In [2]:
convfeats = np.load('conv_feats.npy')
labels = celeb_data['label'].values

X_train = convfeats[partition.partition==0]
X_valid = convfeats[partition.partition==1]
y_train = labels[partition.partition==0]
y_valid = labels[partition.partition==1]

### Create Siamese Model
Siamese model will encode the conv features to a 256 dim vector that will represent the image. 
Kind of a signature of the images. 

The fundamental assumption of this model is that same person should have similar "signatures". 

In [3]:
from keras.layers import Input, Dense, LeakyReLU, Concatenate, Lambda, BatchNormalization
from keras import backend as K
from keras.models import Model, load_model

def embedder(conv_feat_size):
    input = Input((conv_feat_size,), name = 'input')
    normalize = Lambda(lambda x: K.l2_normalize(x, axis=-1), name='normalize')
    x = Dense(512)(input)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dense(128)(x)
    x = normalize(x)
    model = Model(input, x)
    return model
    
def get_siamese_model(conv_feat_size=2048):
    
    input_a = Input( (conv_feat_size,),  name='anchor')
    input_p = Input( (conv_feat_size,),  name='positive')
    input_n = Input( (conv_feat_size,),  name='negative')
    
    emb_model = embedder(conv_feat_size)
    output_a = emb_model(input_a)
    output_p = emb_model(input_p)
    output_n = emb_model(input_n)
    
    merged_vector = Concatenate(axis=-1)([output_a, output_p, output_n])
    model = Model(inputs=[input_a, input_p, input_n],
                  outputs=merged_vector)

    return model

model = get_siamese_model(conv_feat_size = convfeats.shape[1])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


### Create Siamese Model Loss - Triplet Loss

Same person should have 'similar' signatures between his images, whilst new person should have differnet signature.
One way to compare the "similarity" between this signatures (vectors) is to use euclidean distance metric or cosine distance.

I chose to use cosine distance: https://en.wikipedia.org/wiki/Cosine_similarity
But you can easily change it and check if you're getting better results. If you do, let me know :)

Let's define 3 variables:
1. Anchor - The image against which comparisons will be made
2. Positive - Different image of the person in the anchor image
3. Negative - image of a different person

Our loss is essentially: 

###### Loss = Cos_dist(Anchor, Positive) - Cos_dist(Anchor, Negative) + alpha

for more information visit here: https://towardsdatascience.com/siamese-network-triplet-loss-b4ca82c1aec8


In [4]:
def triplet_loss(y_true, y_pred, cosine = True, alpha = 0.2, embedding_size = 128):
    
    ind = int(embedding_size * 2)
    a_pred = y_pred[:, :embedding_size]
    p_pred = y_pred[:, embedding_size:ind]
    n_pred = y_pred[:, ind:]
    if cosine:
        positive_distance = 1 - K.sum((a_pred * p_pred), axis=-1)
        negative_distance = 1 - K.sum((a_pred * n_pred), axis=-1)
    else:
        positive_distance = K.sqrt(K.sum(K.square(a_pred - p_pred), axis=-1))
        negative_distance = K.sqrt(K.sum(K.square(a_pred - n_pred), axis=-1))
    loss = K.maximum(0.0, positive_distance - negative_distance + alpha)
    return loss

### Create image generator for siamese network
The input of the model will be mini-batches of [Anchors, Positives, Negatives] conv features of the images. 

Don't forget - we train on conv features and not on the origianl images

In [5]:
from keras.utils import Sequence

class EmbLoader(Sequence):
    def __init__(self, convfeats, labels, batchSize = 16):
        self.X = convfeats
        self.batchSize = batchSize
        self.y = labels
        self.POS = np.zeros((batchSize, convfeats.shape[1]))
        self.NEG = np.zeros((batchSize, convfeats.shape[1]))
    #gets the number of batches this generator returns
    def __len__(self):
        l,rem = divmod(len(self.y), self.batchSize)
        return (l + (1 if rem > 0 else 0))
    #shuffles data on epoch end
    def on_epoch_end(self):
        a = np.arange(len(self.y))
        np.random.shuffle(a)
        self.X = self.X[a]
        self.y = self.y[a]
    #gets a batch with index = i
    def __getitem__(self, i):
        start = i*self.batchSize
        stop  = (i+1)*self.batchSize
        ancor_labels = self.y[start:stop]
        ancors = self.X[start:stop]
        for k, label in enumerate(ancor_labels):
            pos_idx = np.where(self.y==label)[0]
            neg_idx = np.where(self.y!=label)[0]
            self.NEG[k] = self.X[np.random.choice(neg_idx)]
            pos_idx_hat = pos_idx[(pos_idx<start) | (pos_idx>stop)]
            if len(pos_idx_hat):
                self.POS[k] = self.X[np.random.choice(pos_idx_hat)]
            else:
                # positive examples are within the batch or just 1 example in dataset
                self.POS[k] = self.X[np.random.choice(pos_idx)]
        return [ancors, self.POS[:k+1], self.NEG[:k+1]], np.empty(k+1)

## Launch training

In [6]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
# Compile the model
model.compile(Adam(lr = 0.00005), loss = triplet_loss)

# create generators
train_gen = EmbLoader(X_train, y_train, batchSize = 64)
valid_gen = EmbLoader(X_valid, y_valid, batchSize = 64)
all_gen = EmbLoader(convfeats, labels, batchSize = 64)

checkpoint = ModelCheckpoint('siamese.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
model.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=150, 
                    validation_data=valid_gen, validation_steps=len(valid_gen),
                    workers=12, use_multiprocessing=True, callbacks=[checkpoint])

# train for 50 more epochs with validation data
model.load_weights('siamese.h5')
model.fit_generator(all_gen, steps_per_epoch=len(all_gen), epochs=200, initial_epoch = 150,
                    workers=8, use_multiprocessing=True)
model.save('siamese.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/150
2544/2544 [==============================] - 28s 11ms/step - loss: 0.0403 - val_loss: 0.0399

Epoch 00001: val_loss improved from inf to 0.03989, saving model to siamese.h5
Epoch 2/150
2544/2544 [==============================] - 27s 11ms/step - loss: 0.0391 - val_loss: 0.0395

Epoch 00002: val_loss improved from 0.03989 to 0.03951, saving model to siamese.h5
Epoch 3/150
2544/2544 [==============================] - 31s 12ms/step - loss: 0.0379 - val_loss: 0.0395

Epoch 00003: val_loss improved from 0.03951 to 0.03950, saving model to siamese.h5
Epoch 4/150
2544/2544 [==============================] - 27s 11ms/step - loss: 0.0378 - val_loss: 0.0389

Epoch 00004: val_loss improved from 0.03950 to 0.03894, saving model to siamese.h5
Epoch 5/150
2544/2544 [==============================] - 27s 11ms/step - loss: 0.0378 - val_loss: 0.0401

Epoch 00005: val_loss did not improve from 0.03894
Epoch 6/150
2543/2544 [===================

2544/2544 [==============================] - 31s 12ms/step - loss: 0.0346 - val_loss: 0.0361

Epoch 00043: val_loss did not improve from 0.03546
Epoch 44/150
2544/2544 [==============================] - 28s 11ms/step - loss: 0.0345 - val_loss: 0.0359

Epoch 00044: val_loss did not improve from 0.03546
Epoch 45/150
2544/2544 [==============================] - 25s 10ms/step - loss: 0.0338 - val_loss: 0.0352

Epoch 00045: val_loss improved from 0.03546 to 0.03516, saving model to siamese.h5
Epoch 46/150
2544/2544 [==============================] - 28s 11ms/step - loss: 0.0347 - val_loss: 0.0355

Epoch 00046: val_loss did not improve from 0.03516
Epoch 47/150
2544/2544 [==============================] - 28s 11ms/step - loss: 0.0347 - val_loss: 0.0366

Epoch 00047: val_loss did not improve from 0.03516
Epoch 48/150
2544/2544 [==============================] - 31s 12ms/step - loss: 0.0344 - val_loss: 0.0372

Epoch 00048: val_loss did not improve from 0.03516
Epoch 49/150
2544/2544 [========

 230/2544 [=>............................] - ETA: 1:21 - loss: 0.0331

Process ForkPoolWorker-2478:
Process ForkPoolWorker-2485:
Process ForkPoolWorker-2492:
Process ForkPoolWorker-2479:
Process ForkPoolWorker-2484:
Process ForkPoolWorker-2490:
Process ForkPoolWorker-2474:
Process ForkPoolWorker-2482:
Process ForkPoolWorker-2473:
Process ForkPoolWorker-2495:
Process ForkPoolWorker-2494:
Process ForkPoolWorker-2489:
Process ForkPoolWorker-2481:
Process ForkPoolWorker-2477:
Process ForkPoolWorker-2487:
Process ForkPoolWorker-2476:
Process ForkPoolWorker-2480:
Process ForkPoolWorker-2483:
Process ForkPoolWorker-2486:
Process ForkPoolWorker-2488:
Process ForkPoolWorker-2493:
Process ForkPoolWorker-2496:
Process ForkPoolWorker-2491:
Process ForkPoolWorker-2475:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent c

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "<ipython-input-5-b97b66e36043>", line 27, in __getit



KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt


KeyboardInterrupt: 

In [72]:
full_model = load_model('siamese_xception.h5', compile=False)
# Modify fully-connected layers model with our new one

full_model.layers[3].name = 'conv_model'
model.layers[3].name = 'embedding_model'

conv_model = full_model.layers[3]
fc_model = model.layers[3]

inp_a, inp_p, inp_n = full_model.input
conv_a = conv_model(inp_a)
conv_p = conv_model(inp_p)
conv_n = conv_model(inp_n)
out_a = fc_model(conv_a)
out_p = fc_model(conv_p)
out_n = fc_model(conv_n)

merged_vector = Concatenate(axis=-1, name = 'Concat_3_images')([out_a, out_p, out_n])
final_model = Model(inputs=[inp_a, inp_p, inp_n], outputs=merged_vector)

final_model.save('siamese_xception.h5', include_optimizer=False)

In [ ]:
full_model = load_model('siamese_xception.h5', compile=False)
